<a href="https://colab.research.google.com/github/aidan-lew/junk-drawer/blob/main/finrl_fingpt_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are the recourses to make a venv for colab:
https://stackoverflow.com/questions/74244113/how-to-make-virtual-environment-on-google-colab-that-can-be-saved-on-drive
https://colab.research.google.com/drive/1Vg005uKhgt-ZwyQ11-BGceqY0q07qIzF

In [1]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install torch
!pip install googlesearch-python
!pip install GoogleNews

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-phbbdlln
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-phbbdlln
  Resolved https://github.com/huggingface/peft.git to commit c9b19bb8f33c84ae26609df1b5fb3720ca4089e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86

# Load FinGPT

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
#have to have access to LLama -  via hugging face
### THIS IS ABlE TO RUN ON a T4!!!
# https://github.com/AI4Finance-Foundation/FinGPT/tree/master/fingpt/FinGPT_Forecaster
# https://github.com/AI4Finance-Foundation/FinGPT/blob/master/FinGPT_Inference_Llama2_13B_falcon_7B_for_Beginners.ipynb
### see how the flow works for these, look at the hugging face page dataset for training.



#### note, this is for the forcaster FinGPT model, which is not specifically for sentiment analysis
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,   # optional if you have enough VRAM
)
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

gpt_model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
gpt_model = gpt_model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/40.0M [00:00<?, ?B/s]

# Whole pipeline

## Scraper

In [2]:
from googlesearch import search
from bs4 import BeautifulSoup
import requests

def scrape_news_headlines(ticker, num_articles):
  query = f"{ticker} news" #####find a better url/website because the current one just gets the google web results
  search_results = search(query)
  articles = []
  count = 0
  for url in search_results:
      if count >= num_articles:
          break
      try:
          response = requests.get(url, verify=False, headers={'User-Agent':'Mozilla/5.0'}, #timeout=10
                                  )
          soup = BeautifulSoup(response.text, 'html.parser')
          title = soup.find('title').text if soup.find('title') else 'No title'
          articles.append({'title': title, 'link': url})
          count += 1
      except Exception as e:
          print(f"Error processing {url}: {e}")
  return articles


## Analyzer

In [3]:
def analyze_news_sentiment(articles):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  gpt_model.to(device)  # Move the model to the device

  sentiments = []
  for article in articles:
      inputs = tokenizer(article['title'], return_tensors='pt', max_length=512, truncation=True)
      inputs = {k: v.to(device) for k, v in inputs.items()}  # Move the input tensors to the device
      outputs = gpt_model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
      sentiment = torch.nn.functional.softmax(outputs.logits, dim=1)
      sentiments.append(sentiment.detach().cpu().numpy())  # Move the sentiment tensor back to the CPU before converting to numpy
      #print(sentiments)
  return sentiments

## Predictor

In [4]:
import numpy as np
import yfinance as yf
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd

def make_stock_price_prediction(ticker, sentiments):
    stock_data = yf.download(ticker, period='1d', interval='1m')

    # Extract the positive sentiment score (index 1) from each softmax output
    sentiment_scores = [sentiment[0, 1] for sentiment in sentiments]

    # Create a 1D numpy array from the sentiment scores
    sentiments = np.array(sentiment_scores)

    sentiment_mean = np.mean(sentiments)
    #print(f"Sentiment mean: {sentiment_mean}")

    stock_data['sentiment'] = [sentiment_mean] * len(stock_data)  # Create a new column with the same value repeated for each row
    X = stock_data[['sentiment']]
    y = stock_data['Close']

    #print(f"X shape: {X.shape}")
    #print(f"y shape: {y.shape}")

    # Fit a HistGradientBoostingRegressor model
    fin_model = HistGradientBoostingRegressor()
    fin_model.fit(X, y)

    # Make a prediction
    prediction_input = pd.DataFrame({'sentiment': [sentiment_mean]})
    #print(f"Prediction input shape: {prediction_input.shape}")

    prediction = fin_model.predict(prediction_input)

    #print(f"Prediction: {prediction}")

    return prediction[0]  # Return the scalar prediction


## Pipeline Function

In [5]:
def prediction_pipeline():
  ticker = input('ticker')
  num_articles = int(input('number of articles'))
  articles = scrape_news_headlines(ticker, num_articles)
  sentiments = analyze_news_sentiment(articles)
  prediction = make_stock_price_prediction(ticker, sentiments)
  print('\n' f"Predicted stock price for {ticker}: {prediction}")
  #return ticker
  #return num_articles
  #return prediction

In [10]:
prediction_pipeline()

tickerMicrosoft
number of articles20


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'news.microsoft.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'news.microsoft.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.msn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by HistGradientBoostingRegressor.

# Piece by piece

## Scraper

In [6]:
####### News sources - what about econ data, general news, etc?
#### googlenews API (https://pypi.org/project/GoogleNews/)
### Rueters (https://www.reuters.com/business/)
### https://www.bloomberg.com/
from googlesearch import search
from bs4 import BeautifulSoup
import requests

def scrape_news_headlines(ticker, num_articles):
  query = f"{ticker} news" #####find a better url/website because the current one just gets the google web results
  search_results = search(query)
  articles = []
  count = 0
  for url in search_results:
      if count >= num_articles:
          break
      try:
          response = requests.get(url, verify=False, headers={'User-Agent':'Mozilla/5.0'}, #timeout=10
                                  )
          soup = BeautifulSoup(response.text, 'html.parser')
          title = soup.find('title').text if soup.find('title') else 'No title'
          articles.append({'title': title, 'link': url})
          count += 1
      except Exception as e:
          print(f"Error processing {url}: {e}")
  return articles

In [7]:
ticker = 'AAPL'
num_articles = 5
articles = scrape_news_headlines(ticker, num_articles)
print(articles)
### returns a [{'':}] list of tuples

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110

[{'title': 'AAPL: Apple Inc - Stock Price, Quote and News - CNBC', 'link': 'https://www.cnbc.com/quotes/AAPL'}, {'title': 'Apple Inc. (AAPL) Latest Stock News & Headlines - Yahoo Finance', 'link': 'https://finance.yahoo.com/quote/AAPL/news/'}, {'title': 'Apple (AAPL) News Headlines | Nasdaq', 'link': 'https://www.nasdaq.com/market-activity/stocks/aapl/news-headlines'}, {'title': 'Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance', 'link': 'https://finance.yahoo.com/quote/AAPL/'}, {'title': 'marketwatch.com', 'link': 'https://www.marketwatch.com/investing/stock/aapl'}]


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.marketwatch.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [19]:
### trying to get acctual articles from google lol

####### look at name
from GoogleNews import GoogleNews

def get_news_articles(ticker, num_articles):
    gn = GoogleNews(lang='en')
    gn.search(ticker)
    news_articles = gn.results(num_articles)
    articles = []
    print(news_articles)
    print(articles)
    for article in news_articles:
          soup = BeautifulSoup(news_articles.text, 'html.parser')
          title = soup.find('title').text if soup.find('title') else 'No title'
          articles.append({'title': title, 'link': })
          articles.append({
            'title': article['title'],
        })
          print(articles)

    return news_articles[:num_articles]

ticker = 'AAPL'
num_articles = 5
articles = get_news_articles(ticker, num_articles)
print(articles)

SyntaxError: expression expected after dictionary key and ':' (<ipython-input-19-04b4f9611615>, line 16)

In [20]:
from GoogleNews import GoogleNews
#### changed name
def scrape_news_articles(ticker, num_articles):
    gn = GoogleNews(lang='en')
    gn.search(ticker)
    news_articles = gn.results(num_articles)
    titles = [article['title'] for article in news_articles]
    return titles

ticker = 'AAPL'
num_articles = 5
articles = scrape_news_articles(ticker, num_articles)
print(articles)


### returns a [''] list

['Hands-on with SharePlay Remote Control on iPadOS 18 [Video]', 'Apple Stock: iPhone 16 Sales Likely ‘Stronger For Longer’', "Apple rejected Meta's pursuit of AI partnership: BBG", 'Apple Inc. (AAPL) is Attracting Investor Attention: Here is What You Should Know', "This is Who Owns the Most Apple Stock. Hint: It's Not Tim Cook.", 'Second Apple campus at RTP delayed by up to four years', 'The Morning After: Apple may face another huge EU fine', 'Apple Inc (NASDAQ:AAPL): The Best Undervalued AI Stock to Buy Now?', "No Deal: Apple (AAPL) Won't Integrate Meta AI - TipRanks.com", 'Evercore says AI could help Apple crush 2025 iPhone growth estimates By Investing.com']


## Analyzer

In [26]:
import torch
def analyze_news_sentiment(articles):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  gpt_model.to(device)  # Move the model to the device

  sentiments = []
  for article in articles:
      inputs = tokenizer(article['title'], return_tensors='pt', max_length=512, truncation=True)
      inputs = {k: v.to(device) for k, v in inputs.items()}  # Move the input tensors to the device
      outputs = gpt_model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
      sentiment = torch.nn.functional.softmax(outputs.logits, dim=1)
      sentiments.append(sentiment.detach().cpu().numpy())  # Move the sentiment tensor back to the CPU before converting to numpy
      print(sentiments)
  return sentiments

In [27]:
ticker = 'AAPL'
num_articles = 5
articles = scrape_news_articles(ticker, num_articles)
sentiments = analyze_news_sentiment(articles)

NameError: name 'tokenizer' is not defined

In [ ]:
sentiments = analyze_news_sentiment(articles)

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32)]
[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, .

In [ ]:
sentiments

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
          1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
         [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
          1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
         [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
          2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
         ...,
         [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
          2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
         [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
          2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
         [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
          5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32),
 array([[[6.4145797e-04, 5.0152973e-03, 3.4871415e-04, ...,
          4.5422399e-03, 8.1572525e-02, 1.2902316e-03],
         [9.0460083e-04, 1.3417624e-04, 7.4513872e-07, ...,
          6.7129950e-03, 3.6121567e-03, 3.7188535e-03],
         [1.1063426e-02, 9.1692823e-04, 

## Predictor

In [5]:
import numpy as np
import yfinance as yf
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd

def make_stock_price_prediction(ticker, sentiments):
    stock_data = yf.download(ticker, period='1d', interval='1m')

    # Extract the positive sentiment score (index 1) from each softmax output
    sentiment_scores = [sentiment[0, 1] for sentiment in sentiments]

    # Create a 1D numpy array from the sentiment scores
    sentiments = np.array(sentiment_scores)

    sentiment_mean = np.mean(sentiments)
    print(f"Sentiment mean: {sentiment_mean}")

    stock_data['sentiment'] = [sentiment_mean] * len(stock_data)  # Create a new column with the same value repeated for each row
    X = stock_data[['sentiment']]
    y = stock_data['Close']

    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")

    # Fit a HistGradientBoostingRegressor model
    fin_model = HistGradientBoostingRegressor()
    fin_model.fit(X, y)

    # Make a prediction
    prediction_input = pd.DataFrame({'sentiment': [sentiment_mean]})
    print(f"Prediction input shape: {prediction_input.shape}")

    prediction = fin_model.predict(prediction_input)

    print(f"Prediction: {prediction}")

    return prediction[0]  # Return the scalar prediction


In [ ]:
make_stock_price_prediction(ticker, sentiments)

[*********************100%%**********************]  1 of 1 completed

Sentiment mean: 0.105434849858284
X shape: (99, 1)
y shape: (99,)
Prediction input shape: (1, 1)
Prediction: [209.95151697]


209.9515169702395

## Pipeline Function

In [6]:
def prediction_pipeline():
  ticker = input('ticker')
  num_articles = int(input('number of articles'))
  articles = scrape_news_headlines(ticker, num_articles)
  sentiments = analyze_news_sentiment(articles)
  prediction = make_stock_price_prediction(ticker, sentiments)
  print('\n' f"Predicted stock price for {ticker}: {prediction}")
  return ticker
  return num_articles
  return prediction

In [7]:
prediction_pipeline()

tickerAAPL
number of articles10


NameError: name 'GoogleNews' is not defined

In [ ]:
ticker = 'AAPL'
num_articles = 10
prediction_pipeline(ticker, num_articles)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32)]
[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, .

[*********************100%%**********************]  1 of 1 completed

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32), array([[[1.56065449e-04, 2.30889302e-03, 7.26893340e-05, ...,
         1.96373928e-03, 9.80778132e-03, 2.92713230e-04],
        [2.20087561e-04, 6.15299141e-05, 1.56542086e-07, ...,
         2.90221884e-03, 4.34303569e-04, 8.43691756e-04],
        [3.48229078e-05, 5.06131255e-05, 1.0

209.94394170587714

In [ ]:
ticker = 'AAPL'
num_articles = 10

articles = scrape_news_headlines(ticker, num_articles)
sentiments = analyze_news_sentiment(articles)
prediction = make_stock_price_prediction(ticker, sentiments)

print(f"Predicted stock price for {ticker}: {prediction}")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32)]
[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, .

[*********************100%%**********************]  1 of 1 completed

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32), array([[[1.56065449e-04, 2.30889302e-03, 7.26893340e-05, ...,
         1.96373928e-03, 9.80778132e-03, 2.92713230e-04],
        [2.20087561e-04, 6.15299141e-05, 1.56542086e-07, ...,
         2.90221884e-03, 4.34303569e-04, 8.43691756e-04],
        [3.48229078e-05, 5.06131255e-05, 1.0

# Google News
Scraper & analyzer

In [30]:
from GoogleNews import GoogleNews
#### changed name

def scrape_news_articles(ticker, num_articles):
    gn = GoogleNews(lang='en')
    gn.search(ticker)
    news_articles = gn.results(num_articles)
    titles = [article['title'] for article in news_articles]
    return titles

ticker = 'AAPL'
num_articles = 5
articles = scrape_news_articles(ticker, num_articles)
print(articles)


### returns a [''] list

['Hands-on with SharePlay Remote Control on iPadOS 18 [Video]', 'Apple Stock: iPhone 16 Sales Likely ‘Stronger For Longer’', "Apple rejected Meta's pursuit of AI partnership: BBG", 'Apple Inc (NASDAQ:AAPL): The Best Undervalued AI Stock to Buy Now?', "This is Who Owns the Most Apple Stock. Hint: It's Not Tim Cook.", 'Second Apple campus at RTP delayed by up to four years', 'The Morning After: Apple may face another huge EU fine', 'Evercore says AI could help Apple crush 2025 iPhone growth estimates By Investing.com', "No Deal: Apple (AAPL) Won't Integrate Meta AI - TipRanks.com", "Apple's Misery In The EU: Company Becomes First To Be Charged For Violating DMA"]


In [36]:
import torch
def analyze_news_sentiment(articles):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  #gpt_model.to(device)  # Move the model to the device

  sentiments = []
  for article in articles:
      inputs = tokenizer(article, return_tensors='pt', max_length=512, truncation=True)
      inputs = {k: v.to(device) for k, v in inputs.items()}  # Move the input tensors to the device
      outputs = gpt_model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
      sentiment = torch.nn.functional.softmax(outputs.logits, dim=1)
      sentiments.append(sentiment.detach().cpu().numpy())  # Move the sentiment tensor back to the CPU before converting to numpy
      print(sentiments)
  return sentiments

In [37]:
ticker = 'AAPL'
num_articles = 5
articles = scrape_news_articles(ticker, num_articles)
sentiments = analyze_news_sentiment(articles)

NameError: name 'gpt_model' is not defined

# With Sentiment FinGPT
look at their hugging face (https://huggingface.co/FinGPT/fingpt-sentiment_llama2-13b_lora)


In [28]:
!pip install transformers==4.32.0 peft==0.5.0
!pip install sentencepiece
!pip install accelerate
!pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
  Attempting uninstall: peft
    Found existing installation: peft 0.11.2.dev0
    Uninstalling peft-0.11.2.dev0:
      Successfully uninstalled peft-0.11.2.dev0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.0 MB/s eta 0:00:00


In [38]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0

# Load Models
#base_model = "NousResearch/Llama-2-13b-hf"
#peft_model = "FinGPT/fingpt-sentiment_llama2-13b_lora"
#tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
#tokenizer.pad_token = tokenizer.eos_token
#model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0", load_in_8bit = True,)
#model = PeftModel.from_pretrained(model, peft_model)
#model = model.eval()


base_model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-13b-hf",
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,   # optional if you have enough VRAM
)
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-13b-hf")

gpt_model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-sentiment_llama2-13b_lora')
gpt_model = gpt_model.eval()



# Make prompts
prompt = [
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google ( NASDAQ : GOOG ) Cash advertiser .
Answer: ''',
]

# Generate results
tokens = tokenizer(prompt, return_tensors='pt', padding=True, max_length=512)
res = model.generate(**tokens, max_length=512)
res_sentences = [tokenizer.decode(i) for i in res]
out_text = [o.split("Answer: ")[1] for o in res_sentences]

# show results
for sentiment in out_text:
    print(sentiment)

# Output:
# positive
# neutral
# negative


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/196 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2047: UserWarning: for base_model.model.model.layers.16.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2047: UserWarning: for base_model.model.model.layers.16.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta 

ValueError: We need an `offload_dir` to dispatch this model according to this `device_map`, the following submodules need to be offloaded: base_model.model.model.layers.15, base_model.model.model.layers.16, base_model.model.model.layers.17, base_model.model.model.layers.18, base_model.model.model.layers.19, base_model.model.model.layers.20, base_model.model.model.layers.21, base_model.model.model.layers.22, base_model.model.model.layers.23, base_model.model.model.layers.24, base_model.model.model.layers.25, base_model.model.model.layers.26, base_model.model.model.layers.27, base_model.model.model.layers.28, base_model.model.model.layers.29, base_model.model.model.layers.30, base_model.model.model.layers.31, base_model.model.model.layers.32, base_model.model.model.layers.33, base_model.model.model.layers.34, base_model.model.model.layers.35, base_model.model.model.layers.36, base_model.model.model.layers.37, base_model.model.model.layers.38, base_model.model.model.layers.39, base_model.

# New Section